In [8]:
import torch
import torchvision.transforms as transforms
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from scipy import interpolate
import random
image_path = '/Users/patricknaylor/Desktop/Field_Detection/Images/Masked/'
model_path = '/Users/patricknaylor/Desktop/Field_Detection/data/Models/'
torch.__version__
device = torch.device('mps')



Format Data data to match inputs of model

Mask files need to be transposed to match image shapes

In [9]:

images_paths = list(glob.glob(image_path + '*.jpg'))
scale_percent = 10

images = []
masks = []
for (img) in (images_paths):
    #Load and shrink images
    msk = f'{image_path}{img[59:-4]}.npy'
    image = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    images.append(cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA))

    #Load and interpolate shrink masks
    mask = np.load(msk).T
    mask = mask[25: -25, 25: -25]
    x = np.linspace(0, 1, mask.shape[0])
    y = np.linspace(0, 1, mask.shape[1])
    f = interpolate.interp2d(y, x, mask, kind='linear')

    x2 = np.linspace(0, 1, 36)
    y2 = np.linspace(0, 1, 64)

    mask = f(y2, x2)
    mask = np.expand_dims(mask, axis=2)
    
    masks.append(mask)

print(f'Image size: {image.shape}\nMask size: {mask.shape}')


In [11]:
images_arr = np.stack(images)
masks_arr = np.stack(masks)
images_arr = images_arr.astype(np.uint8)
masks_arr[masks_arr > 0] = 1
masks_arr[masks_arr <= 0] = 0
masks_arr = masks_arr.astype(np.uint8)

Randomly reflect images along the x and y axis.

Most images are oriented with masked are near the top of the image which creates bias.

In [14]:
for i in range(np.shape(images_arr)[0]):
    rand1 = random.randint(1, 2)
    rand2 = random.randint(1, 2)
    temp_arr_im = images_arr[i, :, :, :]
    temp_arr_msk = masks_arr[i, :, :, :]
    if rand1 == 2:
        temp_arr_im = np.flip(temp_arr_im, 0)
        temp_arr_msk = np.flip(temp_arr_msk, 0)
    if rand2 == 2:
        temp_arr_im = np.flip(temp_arr_im, 1)
        temp_arr_msk = np.flip(temp_arr_msk, 1)
    images_arr[i, :, :, :] = temp_arr_im
    masks_arr[i, :, :, :] = temp_arr_msk


Create image dataset and transform images into PyTorch format

In [15]:
class imageDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        mask = self.y[index]
        y = torch.tensor(mask)
        X = self.transform(image)
        return [X, y]

    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()])

Generate dataloaders for training data and two validation sets

Using 1/6 of data for validation

In [17]:
batch_size = 200
transformed_dataset = imageDataset(X=images_arr, y=masks_arr)
val_size = np.shape(masks_arr)[0]//6
train_size = np.shape(masks_arr)[0] - val_size
train_set, val_set = torch.utils.data.random_split(transformed_dataset, [train_size, val_size])
train_dl = DataLoader(train_set, batch_size, shuffle=True)
val1_dl = DataLoader(val_set, 1, shuffle=True)
val2_dl = DataLoader(val_set, val_size, shuffle=True)
print(f'Number of training images: {train_size} \nNumber of validation images: {val_size}')



Number of training images: 2715 
Number of validation images: 543


### Model class

Two convolutional layers with one pooling layer.

Three fully connected linear layers

relu used for activation 


In [18]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding='same')
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 5, padding='same')
        self.fc1 = nn.Linear(int(12*72*128*.25), int(4*72*128*.25))
        self.fc2 = nn.Linear(int(4*72*128*.25), int(2*72*128*.25))       
        self.fc3 = nn.Linear(int(2*72*128*.25), int(1*72*128*.25))
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


Using Binary Cross Entropy with Logits Loss for loss as it supports multi-label classification

Using Stochastic gradient descent

In [19]:
model = Model().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = .9)

In [20]:
num_epochs = 500
n_total_steps = len(train_dl)
for epoch in range(num_epochs):
    running_loss = 0.0
    sum = 0
    for i, (input, target) in enumerate(train_dl):
        input = input.to(device)
        target = torch.flatten(target, 1).to(device)

        optimizer.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, target.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        sum += 1
    if epoch%10 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')



Epoch [1/500], Loss: 0.6932
Epoch [11/500], Loss: 0.6909
Epoch [21/500], Loss: 0.5660
Epoch [31/500], Loss: 0.5770
Epoch [41/500], Loss: 0.2979
Epoch [51/500], Loss: 0.2292
Epoch [61/500], Loss: 0.2509
Epoch [71/500], Loss: 0.2196
Epoch [81/500], Loss: 0.1920
Epoch [91/500], Loss: 0.1721
Epoch [101/500], Loss: 0.1590
Epoch [111/500], Loss: 0.1399
Epoch [121/500], Loss: 0.1466
Epoch [131/500], Loss: 0.1247
Epoch [141/500], Loss: 0.1313
Epoch [151/500], Loss: 0.1320
Epoch [161/500], Loss: 0.1306
Epoch [171/500], Loss: 0.1270
Epoch [181/500], Loss: 0.1118
Epoch [191/500], Loss: 0.1068
Epoch [201/500], Loss: 0.1039
Epoch [211/500], Loss: 0.1050
Epoch [221/500], Loss: 0.0944
Epoch [231/500], Loss: 0.1008
Epoch [241/500], Loss: 0.1050
Epoch [251/500], Loss: 0.0925
Epoch [261/500], Loss: 0.0959
Epoch [271/500], Loss: 0.1000
Epoch [281/500], Loss: 0.0948
Epoch [291/500], Loss: 0.0691
Epoch [301/500], Loss: 0.0864
Epoch [311/500], Loss: 0.0863
Epoch [321/500], Loss: 0.0758
Epoch [331/500], Loss

In [ ]:
with torch.no_grad():
    for (images, labels) in val2_dl:
        images = images.to(device)
        labels = labels.to(device)
        output = model(images)
        loss = criterion(output, torch.flatten(labels,1, 3).float())
        print(loss.item())

0.10999426245689392
0.08652867376804352


In [ ]:
with torch.no_grad():
    for idx, (images, labels) in enumerate(val1_dl):
        images = images.to(device)
        labels = labels.to(device)
        image = images[0,:,:,:]
        mask = labels[0,:,:,:]
        print(mask.size())
        output = model(images)
        output = output.reshape(36, 64)
        plt.subplot(6,3,(idx*3)+1)
        model_output = torch.Tensor.cpu(F.sigmoid(output))
        model_output[model_output>.5] = 1
        model_output[model_output<=.5] = 0
        plt.imshow(model_output, vmin=0, vmax=1)
        plt.colorbar()
        plt.title(idx)
        plt.subplot(6,3,(3*idx)+2)
        plt.imshow(torch.Tensor.cpu(image).permute(1, 2, 0))
        plt.title(idx)
        plt.subplot(6,3,(3*idx)+3)
        plt.imshow(torch.Tensor.cpu(mask))
        plt.colorbar()
        plt.title(idx)
        print(output.size())
        if idx == 5:
            break

NameError: name 'val1_dl' is not defined

In [ ]:
torch.save(model.state_dict(), model_path + 'model_1.pth')